    -hiveQL : http://hortonworks.com/wp-content/uploads/2016/05/Hortonworks.CheatSheet.SQLtoHive.pdf
    -hive function : https://cwiki.apache.org/confluence/display/hive/languagemanual+udf

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

In [2]:
sc = SparkContext('local')

In [3]:
sqlCtx = SQLContext(sc)

In [4]:
emp =[('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept = [('개발',1), ('연구',2),
        ('영업',3),('기획',4) ]

In [5]:
empRdd = sc.parallelize(emp)
empRdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [6]:
empDF1 = empRdd.toDF() #분산 데이터프레임으로 변환
empDF1

DataFrame[_1: string, _2: bigint]

In [7]:
empDF1.show()

+-------+---+
|     _1| _2|
+-------+---+
| 홍길동|  1|
| 이순신|  2|
| 임꺽정|  3|
| 김철수|  3|
|김철수1|  5|
+-------+---+



In [8]:
empDF = sqlCtx.createDataFrame(emp)
empDF

DataFrame[_1: string, _2: bigint]

In [9]:
empDF.show()

+-------+---+
|     _1| _2|
+-------+---+
| 홍길동|  1|
| 이순신|  2|
| 임꺽정|  3|
| 김철수|  3|
|김철수1|  5|
+-------+---+



In [17]:
df = sqlCtx.read.csv('data/crime_in_Seoul.csv', encoding='euc-kr', 
                     header=True, inferSchema=True) #inferSchema : 데이터 타입 자동으로 맞춰줌
                                                    #header와 inferSchema는 필수
df

DataFrame[관서명: string, 살인 발생: int, 살인 검거: int, 강도 발생: int, 강도 검거: int, 강간 발생: int, 강간 검거: int, 절도 발생: string, 절도 검거: string, 폭력 발생: string, 폭력 검거: string]

In [18]:
df.show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|  종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|  혜화서|        3|        2|        5|        4|       96|       63|    1,114|      424|    1,015|      861|
|  용산서|        5|        5|       14|       14|      194|      173|    1,557|      587|    2,050|    1,704|
|  성북서|        2|        2|        2|        1|       86|       

In [19]:
df.printSchema()

root
 |-- 관서명: string (nullable = true)
 |-- 살인 발생: integer (nullable = true)
 |-- 살인 검거: integer (nullable = true)
 |-- 강도 발생: integer (nullable = true)
 |-- 강도 검거: integer (nullable = true)
 |-- 강간 발생: integer (nullable = true)
 |-- 강간 검거: integer (nullable = true)
 |-- 절도 발생: string (nullable = true)
 |-- 절도 검거: string (nullable = true)
 |-- 폭력 발생: string (nullable = true)
 |-- 폭력 검거: string (nullable = true)



In [20]:
empDF.show()

+-------+---+
|     _1| _2|
+-------+---+
| 홍길동|  1|
| 이순신|  2|
| 임꺽정|  3|
| 김철수|  3|
|김철수1|  5|
+-------+---+



In [22]:
testDF = sqlCtx.createDataFrame(emp, ['name', 'deptid'] )
testDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [23]:
testDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- deptid: long (nullable = true)



In [24]:
testDF.toPandas() # = collect / 원데이터 방대할 시 쥬금..조심해서 사용

,name,deptid
0,홍길동,1
1,이순신,2
2,임꺽정,3
3,김철수,3
4,김철수1,5


#### sql문 사용을 위해 sparkDataframe 이름 부여

In [25]:
testDF.createOrReplaceTempView('my') #데이터프레임 이름 지정 / 변수명 헷갈림 주의

In [51]:
# sql = "select * from my"
# sql = "select * from my where deptid=1"
# sql = "select * from my where deptid=1 or deptid=2"
# sql = "select * from my where name='홍길동'"
# sql = "select * from my where deptid in (1,3)"
# sql = "select * from my where deptid >= 2 and deptid <= 4"
# sql = "select * from my where deptid between 2 and 4"
# sql = "select * from my where name like '%김%'"
# sql = "select * from my where name rlike '%김%'" #r : regular expression
# sql = "select * from my where name rlike '^김%'" #김으로 시작
# sql = "select * from my where name rlike '신$'" #신으로 끝남 
# sql = "select * from my where name rlike '[길김]'"
# sql = "select * from my where name rlike '홍길|철수'"
# sql = "select max(deptid) from my"
# sql = "select min(deptid) from my"
# sql = "select sum(deptid) from my"
# sql = "select avg(deptid) from my"
# sql = "select sum(deptid), avg(deptid) from my"
# sql = "select stddev_samp(deptid) as std from my"
# sql = "select deptid+2 from my"
# sql = "select deptid*2 from my"
# sql = "select * from my order by name"
# sql = "select * from my order by name desc"
sql = "select * from my order by name desc limit 3"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|홍길동|     1|
|임꺽정|     3|
|이순신|     2|
+------+------+



#### 연습문제

In [53]:
# name, salary 컬럼명을 가지는 데이터프레임을 만드시요.
data1 = ['홍길동,1000','이순신,2000','임꺽정,3000',
         '김철수,4000','이황,5000','이이,6000']

    1. 급여가 가장높은 name과 salary를 출력하시요
    2. name, salary, tax 를 출력하시요 (tax는 급여에서 세금 3.3을 제한값)
    3. name 에 '철'이 포함된  name,salary 를 출력하시요
    4. salary top 5 인 name, salary 출력하시요
    5. salary  2000  과 4000 사이의 데이터를 출력하시요

In [54]:
ddRdd = sc.parallelize(data1)
ddRdd

ParallelCollectionRDD[162] at readRDDFromFile at PythonRDD.scala:262

In [57]:
ddRdd.collect()

['홍길동,1000', '이순신,2000', '임꺽정,3000', '김철수,4000', '이황,5000', '이이,6000']

In [75]:
dRdd = ddRdd.map(lambda v:v.split(',') ).map(lambda v: (v[0], int(v[1])) )
dRdd.collect()

[('홍길동', 1000),
 ('이순신', 2000),
 ('임꺽정', 3000),
 ('김철수', 4000),
 ('이황', 5000),
 ('이이', 6000)]

In [76]:
DF = dRdd.toDF(['name', 'salary']) #분산 데이터프레임으로 변환
DF

DataFrame[name: string, salary: bigint]

In [77]:
DF.show()

+------+------+
|  name|salary|
+------+------+
|홍길동|  1000|
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
|  이황|  5000|
|  이이|  6000|
+------+------+



In [78]:
DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [79]:
DF.createOrReplaceTempView('my2') 

In [80]:
# 1. 급여가 가장높은 name과 salary를 출력하시요
# sql = "select * from my2 order by salary desc limit 1" -> 동률이면 하나만 출력
sql = "select * from my2 where salary = (select max(salary) from my2)"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+----+------+
|name|salary|
+----+------+
|이이|  6000|
+----+------+



In [82]:
# 2. name, salary, tax 를 출력하시요 (tax는 급여에서 세금 3.3을 제한값)
sql = "select name, salary, salary*(1-0.033) as tax from my2"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+--------+
|  name|salary|     tax|
+------+------+--------+
|홍길동|  1000| 967.000|
|이순신|  2000|1934.000|
|임꺽정|  3000|2901.000|
|김철수|  4000|3868.000|
|  이황|  5000|4835.000|
|  이이|  6000|5802.000|
+------+------+--------+



In [83]:
# 3. name 에 '철'이 포함된  name,salary 를 출력하시요
sql = "select * from my2 where name rlike '철' "
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|김철수|  4000|
+------+------+



In [84]:
# 4. salary top 5 인 name, salary 출력하시요
sql = "select * from my2 order by salary desc limit 5"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|  이이|  6000|
|  이황|  5000|
|김철수|  4000|
|임꺽정|  3000|
|이순신|  2000|
+------+------+



In [85]:
# 5. salary  2000  과 4000 사이의 데이터를 출력하시요
sql = "select * from my2 where salary between 2000 and 4000"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
+------+------+



In [103]:
# +) '정도' 컬럼 생성: 급여가 5000 이상이면 '많음', 3000 이상이면 '보통', 3000 미만이면 '적음'
sql = "select name, salary, case when salary >= 5000 then '많음' when salary >= 3000 then '보통' else '적음' end as `정도` from my2"
sqlDF = sqlCtx.sql(sql)                                                                         #한글 인식 안 돼서 백쿼트로
sqlDF.show()

+------+------+----+
|  name|salary|정도|
+------+------+----+
|홍길동|  1000|적음|
|이순신|  2000|적음|
|임꺽정|  3000|보통|
|김철수|  4000|보통|
|  이황|  5000|많음|
|  이이|  6000|많음|
+------+------+----+



In [104]:
sc.stop()